## Objetivo del proyecto

Construir y desplegar una aplicación web (Streamlit) que, dado un pedido en el momento de la compra, prediga el tiempo estimado de entrega (ETA, en horas) para ayudar a empresas de delivery (por ejemplo, Glovo, Uber Eats, Just Eat o Deliveroo) a gestionar expectativas del cliente y anticipar riesgos operativos.

#### Enfoque del proyecto: empresas de delivery

Aunque el proyecto se inspira en plataformas conocidas, no construimos un modelo para una empresa concreta, porque sus datos internos (pedidos reales, rutas, repartidores, etc.) no suelen ser públicos.

Por eso, hablaremos de empresas de delivery y desarrollaremos un sistema que sea:

* Reutilizable: aplicable a distintas empresas de delivery.

* Escalable: funciona con más datos o con nuevas ciudades/zonas.

* Adaptable: si una empresa aporta datos reales, el modelo se puede reentrenar para ajustarse a su contexto.

**¿Qué estamos construyendo exactamente?**

Un MVP (Producto Mínimo Viable): una primera versión funcional que predice el ETA usando información disponible en el momento de compra.

La app mostrará:

* ETA estimado (horas)

* (Opcional) un semáforo de riesgo comparando el ETA estimado con la fecha prometida del pedido.

**¿Qué es Olist?**

Olist fue una plataforma de comercio electrónico en Brasil que conectaba a tiendas (vendedores) con grandes marketplaces. Los pedidos se gestionaban con información como:

* momento de compra del pedido

* productos incluidos

* forma de pago

* vendedor que envía

* cliente que recibe

* fechas de envío y entrega

* (en algunos casos) reseñas tras la entrega

En resumen: es un entorno realista donde cada pedido tiene origen, destino y un tiempo de entrega.

**¿Por qué usamos datos de Olist en un proyecto para “empresas de delivery”?**

Aunque Olist no es una empresa de reparto, su dataset representa un proceso muy similar:

1. Un pedido se crea en un momento concreto.

2. Existe un origen (vendedor) y un destino (cliente).

3. La entrega tarda un tiempo y, a veces, puede retrasarse.

Esto nos permite entrenar un modelo que aprenda patrones como:

* **estacionalidad**: horas/días/meses con mayor carga

* **complejidad** del pedido: más productos, más peso o varios vendedores

* **rutas aproximadas**: combinaciones ciudad/estado vendedor → cliente con tiempos distintos

Por todos estos motivos, la elección de Olist creemos que nos aportará un buen punto de partida para construir un producto mínimo viable para la predicción de tiempo estimado de entrega en empresas de delivery. 

## Descripción de los datos y elección de variables

En esta sección vamos a revisar cada archivo CSV del dataset de Olist para entender:

* qué información contiene,

* qué variables pueden ayudarnos a predecir el ETA (tiempo estimado de entrega),

* qué variables debemos descartar porque no aportan valor o porque contienen “información del futuro”.

Para que el modelo sea útil en una empresa de delivery, solo usaremos variables que existirían en el momento de la compra.
Variables que ocurren después (como la entrega real o las reseñas) no pueden usarse para predecir, porque sería como “adivinar con la respuesta ya conocida”.


### Estructura que seguiremos en cada CSV

Para que el análisis sea claro y repetible, cada dataset tendrá estos cuatro bloques:

1. **Qué contiene y para qué sirve**

Explicación sencilla de qué representa el CSV dentro del proceso de un pedido.

2. **Variables disponibles**

Lista de columnas del archivo (las veremos en el notebook para confirmarlas).

3. **Selección de variables (decisión)**

* Variables que usaremos (como predictoras o como llaves para unir tablas)

* Variables que NO usaremos (por irrelevancia o por información futura / data leakage)

4. **Por qué tomamos esa decisión**

Cómo esa información ayuda (o por qué no se puede usar).

Después de cada explicación incluiremos celdas de código para:

+ cargar el CSV,

+ ver su estructura (filas/columnas),

+ revisar valores nulos y tipos de dato,

+ y confirmar que tiene sentido para nuestro objetivo

A continuación listamos las columnas de cada CSV para confirmar la estructura del dataset descargado.

In [2]:
import pandas as pd
from pathlib import Path

In [3]:
raw = Path("../data/raw")  

files = ["olist_orders_dataset.csv",
         "olist_order_items_dataset.csv",
         "olist_order_payments_dataset.csv",
         "olist_order_reviews_dataset.csv",
         "olist_customers_dataset.csv",
         "olist_sellers_dataset.csv",
         "olist_products_dataset.csv",
         "olist_geolocation_dataset.csv",
         "product_category_name_translation.csv"]

rows = []
for f in files:
    df = pd.read_csv(raw / f, nrows=5)
    rows.append({"dataset": f,
                 "n_columns": df.shape[1],
                 "columns": ", ".join(df.columns)})

summary = pd.DataFrame(rows)
summary.index = range(1, len(summary) + 1)
summary

,dataset,n_columns,columns
1,olist_orders_dataset.csv,8,"order_id, customer_id, order_status, order_pur..."
2,olist_order_items_dataset.csv,7,"order_id, order_item_id, product_id, seller_id..."
3,olist_order_payments_dataset.csv,5,"order_id, payment_sequential, payment_type, pa..."
4,olist_order_reviews_dataset.csv,7,"review_id, order_id, review_score, review_comm..."
5,olist_customers_dataset.csv,5,"customer_id, customer_unique_id, customer_zip_..."
6,olist_sellers_dataset.csv,4,"seller_id, seller_zip_code_prefix, seller_city..."
7,olist_products_dataset.csv,9,"product_id, product_category_name, product_nam..."
8,olist_geolocation_dataset.csv,5,"geolocation_zip_code_prefix, geolocation_lat, ..."
9,product_category_name_translation.csv,2,"product_category_name, product_category_name_e..."


#### 1. `olist_orders_dataset.csv`

In [4]:
raw = Path("../data/raw")

orders_raw = pd.read_csv(raw / "olist_orders_dataset.csv")
orders_raw.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00


In [5]:
orders_raw.shape

(99441, 8)

In [6]:
orders_raw.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date'],
      dtype='object')

In [ ]:
orders_raw.dtypes

order_id                         object
customer_id                      object
order_status                     object
order_purchase_timestamp         object
order_approved_at                object
order_delivered_carrier_date     object
order_delivered_customer_date    object
order_estimated_delivery_date    object
dtype: object

In [ ]:
(orders_raw.isna().mean()*100).sort_values(ascending=False)


order_delivered_customer_date    0.029817
order_delivered_carrier_date     0.017930
order_approved_at                0.001609
order_id                         0.000000
order_purchase_timestamp         0.000000
order_status                     0.000000
customer_id                      0.000000
order_estimated_delivery_date    0.000000
dtype: float64

In [18]:
orders_raw["order_status"].value_counts(dropna=False)


order_status
delivered      96478
shipped         1107
canceled         625
unavailable      609
invoiced         314
processing       301
created            5
approved           2
Name: count, dtype: int64